# Using huggingface Transformer

In [1]:
!pip install transformers==4.34.0

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch

model_id = "amazon/MistralLite"

## use flash attention2

In [3]:
!pip install flash-attn==2.3.1.post1 --no-build-isolation

In [4]:
!pip install accelerate==0.23.0

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype=torch.bfloat16,
                                             use_flash_attention_2=True,
                                             device_map="auto",)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt = "<|prompter|>What are the main challenges to support a long context for LLM?</s><|assistant|>"

sequences = pipeline(
    prompt,
    max_new_tokens=400,
    do_sample=False,
    return_full_text=False,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"{seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 The main challenges to support a long context LLM include:

1. Data availability: A long context LLM requires a large amount of data to train on, which can be difficult and expensive to collect and annotate.

2. Computational resources: Training a long context LLM requires a lot of computational resources, including powerful hardware and software, which can be costly and difficult to obtain.

3. Model complexity: A long context LLM is likely to be a complex model, which can be difficult to train and optimize.

4. Evaluation: Evaluating the performance of a long context LLM can be challenging, as it may not be clear what metrics to use or how to interpret the results.

5. Human evaluation: A long context LLM may produce outputs that are difficult for humans to understand or interpret, which can make it difficult to evaluate the model's performance.

6. Ethical considerations: A long context LLM may raise ethical concerns, such as the potential for bias or the impact on privacy and secu

In [5]:
with open("../example_long_ctx.txt", "r") as fin:
    task_instruction = fin.read()
    task_instruction = task_instruction.format(
        my_question="please tell me how does pgvector help with Generative AI and give me some examples."
    )
prompt = f"<|prompter|>{task_instruction}</s><|assistant|>"
sequences = pipeline(
    prompt,
    max_new_tokens=400,
    do_sample=False,
    return_full_text=False,
    num_return_sequences=1,
    #eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"{seq['generated_text']}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


 pgvector is an open-source extension for PostgreSQL supported by Amazon Aurora PostgreSQL-Compatible Edition.

You can use pgvector to store, search, index, and query billions of embeddings that are generated from machine learning (ML) and artificial intelligence (AI) models in your database, such as those from Amazon Bedrock (limited preview) or Amazon SageMaker. A vector embedding is a numerical representation that represents the semantic meaning of content such as text, images, and video.

With pgvector, you can query embeddings in your Aurora PostgreSQL database to perform efficient semantic similarity searches of these data types, represented as vectors, combined with other tabular data in Aurora. This enables the use of generative AI and other AI/ML systems for new types of applications such as personalized recommendations based on similar text descriptions or images, candidate match based on interview notes, customer service next best action recommendations based on successful 